# Azure AI Foundry + Semantic Kernel: Code Interpreter Tutorial

A clean, educational example showing how to:
1. Create an Azure AI Foundry agent with code interpreter
2. Wrap it with Semantic Kernel for conversation management  
3. Generate files with the agent
4. Retrieve and save files locally
5. Clean up resources properly

## Prerequisites
- Azure AI Foundry project with agent service enabled
- Environment variables: `PROJECT_ENDPOINT`, `MODEL_DEPLOYMENT_NAME`
- Libraries: `semantic-kernel>=1.37.0`, `azure-ai-agents>=1.2.0b4`

## 1. Import Libraries & Setup

In [30]:
import os
import nest_asyncio
from pathlib import Path
from dotenv import load_dotenv

# Azure credentials
from azure.identity import DefaultAzureCredential

# Azure AI Foundry SDK - Unified approach for agent creation
from azure.ai.projects import AIProjectClient

# Semantic Kernel - SK client creation and agents
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Setup
nest_asyncio.apply()
load_dotenv()

# Create downloads directory
downloads_dir = Path.cwd() / "azure_ai_files" / "downloads"
downloads_dir.mkdir(parents=True, exist_ok=True)

print("✅ Setup complete!")

✅ Setup complete!


## 2. Create Azure AI Foundry Agent

In [31]:
# Create unified client using Semantic Kernel's recommended approach  
credential = DefaultAzureCredential()

# Use AzureAIAgent.create_client() for all Semantic Kernel operations
client = AzureAIAgent.create_client(
    credential=credential, 
    endpoint=os.environ["PROJECT_ENDPOINT"]
)

# Create agent with code interpreter using unified client
agent_definition = await client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="ChartAgent",
    instructions="Create data visualizations and save as files.",
    tools=[{"type": "code_interpreter"}]
)

print(f"✅ Agent created: {agent_definition.id}")

✅ Agent created: asst_tbqLZ7NsPFxYnUzN2ZDvkTHh


## 3. Create Semantic Kernel Wrapper & Thread

In [32]:
# Create Semantic Kernel agent wrapper and thread using the same client
agent = AzureAIAgent(client=client, definition=agent_definition)
thread = AzureAIAgentThread(client=client)

print("✅ Semantic Kernel agent and thread ready")

✅ Semantic Kernel agent and thread ready


## 4. Chat with Agent (Generate Files)

In [33]:
# Ask agent to create a chart
message = "Create a colorful bar chart showing monthly data (Jan=31, Feb=28, Mar=31, Apr=30, May=31, Jun=30) and save as a file."

async for response in agent.invoke(messages=message, thread=thread):
    print(response.content)

print("\n✅ Agent response complete")

import matplotlib.pyplot as plt

# Data
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
values = [31, 28, 31, 30, 31, 30]
colors = ['#FF9999', '#66B3FF', '#99FF99', '#FFD700', '#FFB6C1', '#A9A9F5'] # Colorful palette

# Create Bar Chart
plt.figure(figsize=(8,5))
bars = plt.bar(months, values, color=colors, edgecolor='black')
plt.title('Monthly Data')
plt.xlabel('Months')
plt.ylabel('Value')
plt.ylim([0,35])

# Save Figure
file_path = '/mnt/data/monthly_bar_chart.png'
plt.savefig(file_path, bbox_inches='tight')
plt.close()

file_path
The colorful bar chart showing the monthly data has been created and saved as a file. You can download it using the link below:

[Download monthly_bar_chart.png](sandbox:/mnt/data/monthly_bar_chart.png)

✅ Agent response complete
The colorful bar chart showing the monthly data has been created and saved as a file. You can download it using the link below:

[Download monthly_bar_chart.png](sandbox:/mnt/data/monthly_bar_chart.png)

✅ Agent response comple

## 5. Collect File IDs from Chat Response

In [34]:
# Extract file IDs from the response items
file_ids = []

# Check for any items with file_id attribute
for item in response.items:
    if hasattr(item, 'file_id') and item.file_id:
        file_ids.append(item.file_id)
        print(f"📁 Found file ID in item: {item.file_id}")

# Alternative: Look for FileReferenceContent in the response items
if not file_ids:
    for item in response.items:
        if hasattr(item, 'content_type') and item.content_type == 'file_reference':
            if hasattr(item, 'file_id') and item.file_id:
                file_ids.append(item.file_id)
                print(f"📁 Found file reference: {item.file_id}")


print(f"📋 Collected file IDs: {file_ids}")
print(f"📄 Total files to download: {len(file_ids)}")

# Optional: Fallback to thread messages if no files found in response
if not file_ids:
    print("⚠️ No files found in response. Checking thread messages as backup...")
    messages = project_client.agents.list_messages(thread_id=thread.id, order="asc")
    
    for msg in messages.data:
        if msg.role == "assistant":
            # Check for image files and annotations
            if hasattr(msg, 'image_contents'):
                for img in msg.image_contents:
                    if hasattr(img, 'image_file') and hasattr(img.image_file, 'file_id'):
                        file_id = img.image_file.file_id
                        if file_id not in file_ids:
                            file_ids.append(file_id)
                            print(f"🖼️ Found image file: {file_id}")
            
            if hasattr(msg, 'file_path_annotations'):
                for ann in msg.file_path_annotations:
                    if hasattr(ann, 'file_path') and hasattr(ann.file_path, 'file_id'):
                        file_id = ann.file_path.file_id
                        if file_id not in file_ids:
                            file_ids.append(file_id)
                            print(f"📄 Found file annotation: {file_id}")
    
    print(f"📄 Total files found: {len(file_ids)}")

📁 Found file ID in item: assistant-9Wv7XABsA96noA9QjBzf4R
📋 Collected file IDs: ['assistant-9Wv7XABsA96noA9QjBzf4R']
📄 Total files to download: 1


## 6. Retrieve File Contents & Save Locally

In [35]:
# Retrieve file contents and save locally using unified client approach
for i, file_id in enumerate(file_ids, 1):
    try:
        print(f"📁 Retrieving content for file {i}: {file_id}")
        
        # Use unified client for file operations - handle async generator correctly
        file_content_generator = await client.agents.files.get_content(file_id)
        
        # Collect all byte chunks from the async generator
        file_data = b''
        async for chunk in file_content_generator:
            file_data += chunk
        
        # Save content to local file
        filename = f"chart_{i}_{file_id[:8]}.png"
        file_path = downloads_dir / filename
        
        with open(file_path, 'wb') as f:
            f.write(file_data)
        
        print(f"✅ Downloaded: {file_path}")
        
    except Exception as e:
        print(f"❌ Error retrieving {file_id}: {e}")

print(f"\n🎉 Files downloaded to: {downloads_dir}")

📁 Retrieving content for file 1: assistant-9Wv7XABsA96noA9QjBzf4R
✅ Downloaded: c:\Users\chrismckee\Documents\GitHub\SKNotebooks\azure_ai_files\downloads\chart_1_assistan.png

🎉 Files downloaded to: c:\Users\chrismckee\Documents\GitHub\SKNotebooks\azure_ai_files\downloads
✅ Downloaded: c:\Users\chrismckee\Documents\GitHub\SKNotebooks\azure_ai_files\downloads\chart_1_assistan.png

🎉 Files downloaded to: c:\Users\chrismckee\Documents\GitHub\SKNotebooks\azure_ai_files\downloads


## 7. Cleanup Resources

In [36]:
# Clean up Azure resources with proper None checks
try:
    # Delete thread if it exists
    if thread is not None:
        await thread.delete()
        print("✅ Thread deleted")
    else:
        print("ℹ️ Thread already cleaned up")
        
    # Delete agent using unified client
    if client is not None and agent_definition is not None:
        await client.agents.delete_agent(agent_definition.id)
        print("✅ Agent deleted")
    else:
        print("ℹ️ Agent already cleaned up")
        
    # Close unified client if it exists
    if client is not None:
        await client.__aexit__(None, None, None)
        print("✅ Client closed")
        
except Exception as e:
    print(f"⚠️ Cleanup error (continuing): {e}")

# Reset variables to None
thread = None
agent = None
client = None
agent_definition = None
file_ids = None

print("✅ All resources cleaned up!")
print("💡 Downloaded files preserved locally")

✅ Thread deleted
✅ Agent deleted
✅ Client closed
✅ All resources cleaned up!
💡 Downloaded files preserved locally
✅ Agent deleted
✅ Client closed
✅ All resources cleaned up!
💡 Downloaded files preserved locally
